Lo de lectura de datos puede quedarse así, porque se crea una instancia virtual, por lo que sólo funciona para saber que existen las imágenes de datos. Posteriormente, se ejecutan los comandos para hacer el llamado a la arquitectura. Luego, se hace el transfer learning, en donde se pasan los pesos. Dichos pesos se congelan para que no haya pérdida de información. Finalmente se hace trabajo con el dataset personalizado (Custom data)

#Lectura de datos

In [ ]:
from google.colab import drive
import pandas as pd

In [ ]:
drive.mount('/content/drive')

In [ ]:
in_dir = 'drive/MyDrive/RIIA_Hack/'
in_file = 'JusticIA_DatosTranscripciones.csv'

df = pd.read_csv(in_dir+in_file)
df.tail()

In [ ]:
import PIL

from glob import glob
from random import sample

def get_width_and_height(file_name):
    image = PIL.Image.open(file_name)
    return image.size
    
files = glob(in_dir+'/*/*.*')
files = sample(files, 20)

for file_name in files:
    width, height = get_width_and_height(file_name)
    print(width, height, file_name)

In [ ]:
df_long = df[df['Texto'].str.len()>2000]

for dir, fil in zip(df_long['Conjunto'].values, df_long['NombreArchivo'].values):
    file_name = in_dir + dir + '/' + fil
    width, height = get_width_and_height(file_name)
    print(width, height, file_name)

#Learning Transfer

Usando la arquitectura de yolov5 (https://github.com/ultralytics/yolov5) hacemos la transferencia de pesos y de la arquitectura.

In [ ]:
!git clone https://github.com/ultralytics/yolov5
!cd yolov5
!pip install -r requirements.txt

In [ ]:
import torch
# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # or yolov5m, yolov5l, yolov5x, custom
# Images
img = 'https://ultralytics.com/images/zidane.jpg'  # or file, Path, PIL, OpenCV, numpy, list
# Inference
results = model(img)
# Results
results.print()  # or .show(), .save(), .crop(), .pandas(), etc.

In [ ]:
!python detect.py --source 0  # webcam
                            file.jpg  # image 
                            file.mp4  # video
                            path/  # directory
                            path/*.jpg  # glob
                            'https://youtu.be/NUsoVlDFqZg'  # YouTube
                            'rtsp://example.com/media.mp4'  # RTSP, RTMP, HTTP stream

In [ ]:
$ python train.py --data coco.yaml --cfg yolov5s.yaml --weights '' --batch-size 64
                                         yolov5m                                40
                                         yolov5l                                24
                                         yolov5x                                16

#Frozen Layers

<h3>https://github.com/ultralytics/yolov5/issues/131</h3>

In [ ]:
 # Freeze 
 freeze = [f'model.{x}.' for x in range(freeze)]  # layers to freeze 
 for k, v in model.named_parameters(): 
     v.requires_grad = True  # train all layers 
     if any(x in k for x in freeze): 
         print(f'freezing {k}') 
         v.requires_grad = False 

In [ ]:
# YOLOv5 backbone 
 backbone: 
   # [from, number, module, args] 
   [[-1, 1, Focus, [64, 3]],  # 0-P1/2 
    [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4 
    [-1, 3, BottleneckCSP, [128]], 
    [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8 
    [-1, 9, BottleneckCSP, [256]], 
    [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16 
    [-1, 9, BottleneckCSP, [512]], 
    [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32 
    [-1, 1, SPP, [1024, [5, 9, 13]]], 
    [-1, 3, BottleneckCSP, [1024, False]],  # 9 
   ] 
  
 # YOLOv5 head 
 head: 
   [[-1, 1, Conv, [512, 1, 1]], 
    [-1, 1, nn.Upsample, [None, 2, 'nearest']], 
    [[-1, 6], 1, Concat, [1]],  # cat backbone P4 
    [-1, 3, BottleneckCSP, [512, False]],  # 13 
  
    [-1, 1, Conv, [256, 1, 1]], 
    [-1, 1, nn.Upsample, [None, 2, 'nearest']], 
    [[-1, 4], 1, Concat, [1]],  # cat backbone P3 
    [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small) 
  
    [-1, 1, Conv, [256, 3, 2]], 
    [[-1, 14], 1, Concat, [1]],  # cat head P4 
    [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium) 
  
    [-1, 1, Conv, [512, 3, 2]], 
    [[-1, 10], 1, Concat, [1]],  # cat head P5 
    [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large) 
  
    [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5) 
   ] 

#Training Custom Data

<h3>https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data</h3>

In [ ]:
# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: ../datasets/coco128  # dataset root dir
train: images/train2017  # train images (relative to 'path') 128 images
val: images/train2017  # val images (relative to 'path') 128 images
test:  # test images (optional)

# Classes
nc: 80  # number of classes
names: [ 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
         'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
         'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
         'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
         'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
         'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
         'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
         'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
         'hair drier', 'toothbrush' ]  # class names